In [31]:
import MySQLdb
import node_edge_gen as neg
import networkx as nx

In [32]:
#filename = 'map_extrasmall.osm'
#int_string, ways, nodes = neg.extract_intersections(filename, verbose= False)
#edges = ways_to_edge(ways)
#neg.plot_nodes_edges(nodes,edges)

#import from mySQL



# DEFINE SQL LOG IN 
HOST = 'localhost'
USER = 'root'
PASSWD = ''
DATABASE = 'bk_map'

#CONNECT TO MySQL
db_connect = MySQLdb.connect(
    host = HOST,
    user = USER,
    passwd = PASSWD,
    db = DATABASE)

cur = db_connect.cursor()

In [33]:
import networkx as nx

In [34]:
#SQL graph into nx #MAKE THIS SMALLER BASED ON THE INPUT RANGE
cur.execute('''SELECT nodeid, latitude, longitude FROM bk_nodes
                ''')  
nodes_sql = cur.fetchall()

cur.execute('''SELECT edgeid, node1, node2 FROM bk_edges
                ''')  
edges_sql = cur.fetchall()

#cursor.close()
#db_connect.close()


In [35]:
#make dictionary for nodes and edges
node_dic = {}
for node in nodes_sql:
    node_dic[node[0]] = node[1:] 

print len(edges_sql)
edge_dic = {}
count = 0
for edge in edges_sql:
    count += 1
    edge_dic[edge[0]]= edge[1:]
    
print count

3951
3951


In [36]:
print 'nodes', len(node_dic)
print 'edges', len(edge_dic)

nodes 2473
edges 3951


In [37]:
G = nx.Graph()
n_count = 0
for d in node_dic:
    n_count += 1
    G.add_node(d)
e_count = 0
for e in edge_dic:
    e_count += 1
    G.add_edge(edge_dic[e][0],edge_dic[e][1],weight = distance_btw_nodes(edge_dic[e][0],edge_dic[e][1],node_dic))

print n_count, G.number_of_nodes()
print e_count, G.number_of_edges()

2473 2473
3951 3922


In [ ]:
import matplotlib.pylab as plt
nx.draw(G, node_dic)
plt.show()

def plot_nodes_edges(nodes,edges):
    for d in nodes:
        plt.scatter(nodes[d][0],nodes[d][1])
    

    for w in edges:
        node_list = []

        for i in edges[w]:
            if i in nodes: 
                node_list.append(nodes[i]) 
            
        plt.plot(*zip(*node_list))
    plt.show()


#plot_nodes_edges(node_dic, edge_dic)

In [38]:
from geopy.distance import great_circle
from geopy.distance import vincenty

In [39]:
def nearest_node(lat_input, lon_input):
    dis_near=0.05
    find_nearby_nodes = ('''SELECT nodeid, latitude, longitude FROM bk_nodes 
                WHERE latitude BETWEEN %s AND %s
                AND longitude BETWEEN %s AND %s
                ''')
    node_range = (lat_input-dis_near, lat_input+dis_near, lon_input-dis_near, lon_input+dis_near)

    cur.execute(find_nearby_nodes,node_range)  
    rows = cur.fetchall()
    if len(rows) == 0:
        return 'NULL'
    #for row in rows:
    else:
        dist = [rows[0][0],great_circle((lat_input,lon_input),rows[0][1:3]).miles ]
        for row in rows[1:]:
            node_dist =  great_circle((lat_input,lon_input),row[1:3]).miles
            if dist[1] > node_dist:
                dist = [row[0],node_dist]
        return dist[0]

In [40]:
start = nearest_node(40.6868, -73.9824)

In [41]:
end = nearest_node(40.6848, -73.9918)

In [42]:
nx.has_path(G,start,end)

True

In [ ]:
cur.close()
db_connect.close()

In [ ]:
nx.shortest_path(G,source = start, target = end, weight='yes')

In [ ]:
nx.dijkstra_path(G,start,end)

In [ ]:
def distance_btw_nodes(node1,node2,node_dic):
    return great_circle(node_dic[node1], node_dic[node2]).miles

In [ ]:
print distance_btw_nodes('42517476','42466447',node_dic)